In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup

In [2]:
baseurl = 'https://weather.com/mn-MN/weather/today/l/MGXX0003:1:MG?Goto=Redirected'
app_list = []
for i in range(1,59): 
  url = baseurl + str(i)
  #print(url)
  #print(url)
  response = requests.get(url)
  if response.status_code != 200:
   print(response.status_code)
   print('error ',url)
   continue
  print(i)
  soup = BeautifulSoup(response.text,"html.parser")
  li_list = soup.find_all("li", {"class": "announcement-container"})
  for li in li_list:
    a = li.find('a')
    appartment_url = 'https://www.unegui.mn'+a['href']
    #print(appartment_url)
    app_list.append(appartment_url)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


In [3]:
print(len(app_list))
app_set = set(app_list)
print(len(app_set))

0
0


In [4]:
class Appartment:
  def __init__(self, title, location, price, space, url):
    self.title = title
    self.location = location
    self.price = price
    self.space = space
    self.url = url

In [5]:
def findFeature(li_list, header):
  ret = 'NA'
  for li in li_list:
    text = li.text.strip()
    if text.startswith(header):
      return text[len(header) + 1:]
  return ret

In [6]:
it = 0
scrappedData=[]
for url in app_set: 
  #print(url)
  it += 1
 
  response = requests.get(url)
  if response.status_code != 200:
   print(response.status_code)
   print('error ',url)
   continue
  soup = BeautifulSoup(response.text,"html.parser")
  title = soup.find("h1", {"class": "title-announcement"}).text.strip()
  meta_data_div = soup.find("div", {"class": "announcement-price__cost"})
  meta_price = meta_data_div.find('meta', {'itemprop': 'price'})['content']
  li_class = soup.find_all("li")
  location = findFeature(li_class,'Байршил:')
  space = findFeature(li_class,'Талбай:')
  elevation = findFeature(li_class,'Хэдэн давхарт:')
  year = findFeature(li_class,'Ашиглалтанд орсон он:')
  district = findFeature(li_class,'Дүүрэг:')
  leasing = findFeature(li_class,'Лизингээр авах боломж:')
  isDone = findFeature(li_class,'Барилгын явц:')

  temp = [title,meta_price,location, space,elevation,year,district,leasing,isDone]
  scrappedData.append(temp)
  if it%31==0:
    print(temp)
print(scrappedData)

[]


In [7]:
import csv

# for x in scrappedData:
#   if float(x[1])<20000000:
#     x[1]=(float(x[1])*float(x[3].split(" ")[0]))

# for idx,x in enumerate(scrappedData):
#     print(x)
with open('output.tsv', 'w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    for x in scrappedData:
      tsv_output.writerow(x)